<a href="https://colab.research.google.com/github/clerfayt28/Pitonchik/blob/main/Exam3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from torch.utils.data import Dataset,DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
import struct
import sys
from array import array
from os import path

train_dataset = torchvision.datasets.MNIST(root='./cache', train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root='./cache', train=False, download=True)

def read(dataset):
    if dataset == "training":
        path_img = "./cache/MNIST/raw/train-images-idx3-ubyte"
        path_lbl = "./cache/MNIST/raw/train-labels-idx1-ubyte"
    elif dataset == "testing":
        path_img = "./cache/MNIST/raw/t10k-images-idx3-ubyte"
        path_lbl = "./cache/MNIST/raw/t10k-labels-idx1-ubyte"
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    with open(path_lbl, 'rb') as f_lable:
        __, size = struct.unpack(">II", f_lable.read(8))
        lbl = array("B", f_lable.read())

    with open(path_img, 'rb') as f_img:
        __, size, nrows, ncols = struct.unpack(">IIII", f_img.read(16))
        img = array("B", f_img.read())

    return lbl, img, size, nrows, ncols

def write_dataset(labels, data, size, rows, cols, output_dir):
    classes = {i: f"class{i}" for i in range(10)}

    output_dirs = [path.join(output_dir, classes[i]) for i in range(10)]

    for dir in output_dirs:
        dir = dir.replace("\\", "/")
        if not path.exists(dir):
            os.mkdir(dir)

    for (i, label) in enumerate(labels):
        output_filename = path.join(output_dirs[label], str(i) + ".jpg")
        print("Writing " + output_filename)

        with open(output_filename, "wb") as h:
            data_i = [
                data[ (i*rows*cols) + j*rows : (i*rows*cols) + (j+1)*rows ] for j in range(rows)
            ]
            data_array = np.asarray(data_i)

            im = Image.fromarray(data_array)


            im.save(output_filename)

for dataset in ["training", "testing"]:
    write_dataset(*read(dataset), path.join("data2", dataset))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3883287.54it/s]


Extracting ./cache/MNIST/raw/train-images-idx3-ubyte.gz to ./cache/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 159731.02it/s]


Extracting ./cache/MNIST/raw/train-labels-idx1-ubyte.gz to ./cache/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1508998.25it/s]


Extracting ./cache/MNIST/raw/t10k-images-idx3-ubyte.gz to ./cache/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4031857.94it/s]


Extracting ./cache/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./cache/MNIST/raw



FileNotFoundError: [Errno 2] No such file or directory: 'data2/training/class0'

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, dir_path):
        self.dir_path = dir_path

        self.len_dataset = 0
        self.data_list = []

        data_tree = os.walk(dir_path)
        for path_dir, dir_list, file_list in data_tree:
            if path_dir == dir_path:
                self.classes = dir_list
                self.class_to_index = {
                    class_name: i for i, class_name in enumerate(self.classes)
                }
                continue

            current_class = path_dir.split('\\')[-1]

            for name in file_list:
                file_path = os.path.join(path_dir, name)
                self.data_list.append((file_path, self.class_to_index[current_class]))
            self.len_dataset += len(file_list)

In [5]:
    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        image = np.array(Image.open(file_path))
        return image, target

In [6]:
dataset_images = CustomImageDataset("data2/training")
dataloader_images = DataLoader(dataset=dataset_images, batch_size=64, shuffle=True)

TypeError: object of type 'CustomImageDataset' has no len()